In [4]:
from datasets import load_dataset
dataset = load_dataset(
    "explodinggradients/amnesty_qa",
    "english_v3",
    trust_remote_code=True
)

Repo card metadata block was not found. Setting CardData to empty.


In [5]:
from ragas import EvaluationDataset

eval_dataset = EvaluationDataset.from_hf_dataset(dataset["eval"])

In [9]:
dataset["eval"][0]

{'user_input': 'What are the global implications of the USA Supreme Court ruling on abortion?',
 'reference': "The global implications of the USA Supreme Court ruling on abortion are significant. The ruling has led to limited or no access to abortion for one in three women and girls of reproductive age in states where abortion access is restricted. These states also have weaker maternal health support, higher maternal death rates, and higher child poverty rates. Additionally, the ruling has had an impact beyond national borders due to the USA's geopolitical and cultural influence globally. Organizations and activists worldwide are concerned that the ruling may inspire anti-abortion legislative and policy attacks in other countries. The ruling has also hindered progressive law reform and the implementation of abortion guidelines in certain African countries. Furthermore, the ruling has created a chilling effect in international policy spaces, empowering anti-abortion actors to undermine

In [10]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, SemanticSimilarity
from ragas import evaluate

In [58]:
from typing import Any, List, Optional
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
import google.generativeai as genai

from langchain_core.language_models.llms import LLM

class GeminiLLM(LLM):
    """custom model pakai gemini"""

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")
        
        llm = genai.GenerativeModel(
            model_name="tunedModels/gemini-welllahh-zerotemp-lrfv-3536", 
        )  # buat jawab pertanyaan medis

        ans = (
            llm.generate_content(prompt, generation_config={"temperature": 0.2})
            .candidates[0]
            .content.parts[0]
            .text
        )
        return ans

    @property
    def _llm_type(self) -> str:
        """Get the type of language model used by this chat model. Used for logging purposes only."""
        return "gemini-welllahh-zerotemp-lrfv-3536"


In [59]:
gemini = GeminiLLM()


In [61]:
print(gemini.invoke("apa faktor resiko penyakit kanker paru-paru?"))

Faktor risiko kanker paru-paru meliputi:

* Merokok: Merokok adalah faktor risiko kanker paru-paru yang paling umum. Merokok dapat menyebabkan kanker paru-paru, dan orang yang merokok lebih banyak dan lebih lama memiliki risiko lebih tinggi untuk terkena kanker paru-paru.
* Paparan asap rokok: Orang yang terpapar asap rokok yang dihembuskan oleh perokok memiliki risiko lebih tinggi untuk terkena kanker paru-paru.
* Paparan radon: Radon adalah gas radioaktif yang dapat menyebabkan kanker paru-paru. Radon dapat ditemukan di rumah dan bangunan lainnya.
* Paparan asbes: Asbes adalah mineral yang dapat menyebabkan kanker paru-paru. Asbes dapat ditemukan di beberapa tempat kerja, seperti tambang dan pabrik.
* Paparan bahan kimia lainnya: Beberapa bahan kimia lainnya dapat menyebabkan kanker paru-paru, seperti arsenik, kromium, dan nikel.
* Riwayat keluarga kanker paru-paru: Orang yang memiliki anggota keluarga yang terkena kanker paru-paru memiliki risiko lebih tinggi untuk terkena kanker pa

In [ ]:
from typing import Any, AsyncIterator, Dict, Iterator, List, Optional

from langchain_core.callbacks import (
    AsyncCallbackManagerForLLMRun,
    CallbackManagerForLLMRun,
)
from langchain_core.language_models import BaseChatModel, SimpleChatModel
from langchain_core.messages import AIMessageChunk, BaseMessage, HumanMessage
from langchain_core.outputs import ChatGeneration, ChatGenerationChunk, ChatResult
from langchain_core.runnables import run_in_executor


class CustomChatModelAdvanced(BaseChatModel):
    """
    """

    model_name: str
    """The name of the model"""
   
    def _generate(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> ChatResult:
        """Override the _generate method to implement the chat model logic.

        This can be a call to an API, a call to a local model, or any other
        implementation that generates a response to the input prompt.

        Args:
            messages: the prompt composed of a list of messages.
            stop: a list of strings on which the model should stop generating.
                  If generation stops due to a stop token, the stop token itself
                  SHOULD BE INCLUDED as part of the output. This is not enforced
                  across models right now, but it's a good practice to follow since
                  it makes it much easier to parse the output of the model
                  downstream and understand why generation stopped.
            run_manager: A run manager with callbacks for the LLM.
        """
        # Replace this with actual logic to generate a response from a list
        # of messages.
        last_message = messages[-1]
        tokens = gemini.invoke(last_message)
        message = AIMessage(
            content=tokens,
            additional_kwargs={},  # Used to add additional payload (e.g., function calling request)

        )
        ##

        generation = ChatGeneration(message=message)
        return ChatResult(generations=[generation])

    @property
    def _llm_type(self) -> str:
        """Get the type of language model used by this chat model."""
        return "gemini-welllahh-zerotemp-lrfv-3536"



In [28]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API key here")
os.environ["GEMINI_API_KEY"] =  os.environ["GOOGLE_API_KEY"]


In [39]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from ragas.embeddings import LangchainEmbeddingsWrapper


embed = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
evaluator_embeddings = LangchainEmbeddingsWrapper(embed)

In [40]:
from ragas.llms import LangchainLLMWrapper
evaluator_llm = LangchainLLMWrapper(gemini)

LangchainLLMWrapper(run_config=RunConfig(timeout=180, max_retries=10, max_wait=60, max_workers=16, exception_types=(<class 'Exception'>,), log_tenacity=False, seed=42), multiple_completion_supported=False)

In [41]:
metrics = [
    LLMContextRecall(llm=evaluator_llm), 
    FactualCorrectness(llm=evaluator_llm), 
    Faithfulness(llm=evaluator_llm),
    SemanticSimilarity(embeddings=evaluator_embeddings)
]
results = evaluate(dataset=eval_dataset, metrics=metrics)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable